# Streaming Availability

## Instructions

### Search Basic (Free)
https://rapidapi.com/movie-of-the-night-movie-of-the-night-default/api/streaming-availability/

Searches through the catalog of an OTT service in a particular country for a particular type of content (movie or series). Allows filtering the result by genre, keywords, and/or language. Returns paginated result with 8 items per page. For each item, returns information like title, IMDb ID, TMDb ID, IMDb rating, link to shows’ pages in streaming services, cast, runtime, poster, and much more.	country – Mandatory parameter of type String. Specifies the country to be searched.

service – Mandatory parameter of type String. Specifies the service to be searched. Possible values are Netflix, prime, Disney, hbo, hulu, peacock, paramount, starz, showtime, apple, mubi.

type – Mandatory parameter of type String. Specifies the type of content that needs to be searched. Possible values are movie/series.

genre – Optional parameter of type number corresponding to genre id. When specified, the endpoint filters the results by this genre. The genre id can be obtained from the genres endpoint.

page – Optional parameter of type number corresponding to a page number. Specifies the position from where the results need to be returned. It can be used for pagination.

keyword – Optional parameter of type String corresponding to a keyword to be searched. When specified, the endpoint filters the results by this keyword.

language – Optional parameter of the type String corresponding to the original language of the show. When specified, the endpoint filters the results by this language.

### List of Supported Countries
Argentina     Australia     Austria     Azerbaijan  Belgium     Brazil     Bulgaria     Canada     Chile     Colombia     Croatia     Cyprus     Czech Republic     Denmark     Ecuador     Estonia     Finland     France     Germany     Greece     Hong Kong     Hungary     Iceland     India     Indonesia     Ireland     Israel     Italy     Japan     Lithuania     Malaysia     Mexico     Moldova     Netherlands     New Zealand     North Macedonia     Norway     Panama     Peru     Philippines     Poland     Portugal     Romania     Russia     Serbia     Singapore     South Africa     South Korea     Spain     Sweden     Switzerland     Thailand     Turkey     Ukraine     United Emirates     United Kingdom     United States     Vietnam
    
    
 ['AR',
 'AU',
 'AT',
 'AZ',
 'BE',
 'BR',
 'BG',
 'CA',
 'CL',
 'CO',
 'HR',
 'CY',
 'DK',
 'EC',
 'EE',
 'FI',
 'FR',
 'DE',
 'GR',
 'HK',
 'HU',
 'IS',
 'IN',
 'ID',
 'IE',
 'IL',
 'IT',
 'JP',
 'LT',
 'MY',
 'MX',
 'NZ',
 'NO',
 'PA',
 'PE',
 'PL',
 'PT',
 'RO',
 'RS',
 'SG',
 'ZA',
 'ES',
 'SE',
 'CH',
 'TH',
 'TR',
 'UA',
 'CZ',
 'MD',
 'NL',
 'MK',
 'PH',
 'RU',
 'KR',
 'AE',
 'GB',
 'US',
 'VN']

### streaming services
 
20 streaming services: Netflix, Amazon Prime Video, Hulu, Disney+, HBO Max, Peacock, Paramount+, Starz, Showtime, Apple TV+, Mubi, Stan, Now, Crave, All 4, BBC iPlayer, BritBox, Hotstar, Zee5 and Curiosity Stream

## Imports and Pandas settings

In [ ]:
import pandas as pd
import json
import requests
import re
import webbrowser
import os 
pd.options.display.max_columns = 999
pd.options.display.max_rows = 999
pd.set_option('display.max_colwidth', 100)

## Functions

In [ ]:
def search_movie_series():
    while True:
        movie_series  = input("Is it a movie or series?").lower()
        if movie_series.startswith("m"):
            movie_series = "movie"
            break
        elif movie_series.startswith("s"):
            movie_series = "series"
            break
        else:
            continue 
    
    return movie_series

In [ ]:
def search_input():
    keyword  = str(input("Search for: ").lower())
    
    return keyword

In [ ]:
# Get the Data
def colect_data(service, country, movie_series, keyword, api):
    """Colects data"""
    
    # URL, key and headers for the API
    url = "https://streaming-availability.p.rapidapi.com/search/basic"
    headers = {
                "X-RapidAPI-Key": api,
                "X-RapidAPI-Host": "streaming-availability.p.rapidapi.com"
                }

    json_lst = [] # Prepare json file list
    
    for ser in service: # Goes through all services in list
        for con in country: # Goes through all the countries in the list
            
            querystring = {"country": con, # from list
                           "service": ser, # from list
                           "type": movie_series, # from input
                           "keyword": keyword,# from input
                           }
            response = requests.request("GET", url, headers=headers, params=querystring) # Get the data
    
            if response.status_code != 200: # Check if it was successful
                continue
            else:
                data = response.json() # Turn response in to json file.
                json_lst.append(data)  # Add to a list of json files
    
    # Transfor the data in to a Datafarme 
    df = pd.concat([pd.DataFrame(json["results"]) for json in json_lst]).reset_index(drop=True)
    
    return df # Return Dataframe


In [ ]:
def clean_data_service(df_original):
    """Clean the data service column"""
    # Create new df with links to service
    df_service = pd.concat([pd.DataFrame.from_dict(dic) for dic in df_original["streamingInfo"]]).reset_index(drop=True)
    
    
    # Turn link dict in to strings
    df_service.fillna("Not available", inplace=True) # Fill nan with string "Not available"
    for key in df_service.keys(): # Get dict from df
        for i in range(0, len(df_service)): # Get index num
            if df_service[key][i] == "Not available": # Skip if no link is available
                continue
            else:
                df_service[key][i] = df_service[key][i]["link"] # Replace dict with string 
    
    df_merged = pd.concat([df_original, df_service], axis=1) # merge the df  
    
    return df_merged

In [ ]:
def add_missing_service(df, service_lst):
    """adds missing services to df"""
    for service in service_lst: # Adds missing service to df
        if service not in df.columns:
            df[service] = "Not available"
    
    return df

In [ ]:
def add_country(df_country):
    """Add country to df"""
    df_country["country"] = "" # Add country column to df
    for i in range(0,len(df_country)): # Add country to column
        df_country["country"][i] = list(df_country["streamingInfo"][i][list(df_country["streamingInfo"][i].keys())[0]])[0]
 
    return df_country

In [ ]:
def clean_df(df):
    """Reduce number of columns"""
    # Reduce number of columns
    df.set_index(["title"], inplace=True)
    df = df[["country", "netflix", "prime", "hbo", "disney","imdbRating","runtime","originalLanguage","overview"]]
    return df


In [ ]:
def df_styler(df):
    """Add caption and change color when hovering over row"""
    
    # Caption size, color and hover color
    styles_caption = [dict(selector="caption",
            props=[("text-align", "center"),
                     ("font-size", "200%"),
                    ("color", 'black'),
                  ('background-color', 'Lightblue')]),
                    dict(selector="tr:hover",
            props=[('background-color', 'Lightblue')])]
    
    # Apply changes   
    df = df.style.set_caption("Stream Availability").set_table_styles(styles_caption)
    return df

In [ ]:
def convert_to_HTML(df):
    """Convert to HTML an display in browser"""
    # convert dataframe to html
    html = df.to_html()
    # save file
    text_file = open("df.html", "w")
    text_file.write(html)
    text_file.close()
    # open in browser
    webbrowser.open(os.getcwd()+ "\\df.html")

## Program

### Lists

In [ ]:
##########################
## TO LONG FOR FREE API ##
##vvvvvvvvvvvvvvvvvvvvvv##
#
# service_lst = ["netflix", "prime", "disney", "hbo", "hulu", "peacock", 
#                "paramount", "starz", "showtime", "apple", "mubi", "Stan",
#               "Now", "Crave", "All4", "BBC" "iPlayer", "BritBox", "Hotstar",
#               "Zee5","CuriosityStream"]
#
# country_code_lst =['CZ', 'MD', 'NL', 'MK', 'PH', 'RU', 'KR', 'AE', 'GB', 'US', 'VN', 'AR', 'AU', 'AT', 'AZ', 'BE', 'BR',
#                   'BG', 'CA', 'CL', 'CO', 'HR', 'CY', 'DK', 'EC', 'EE', 'FI', 'FR', 'DE', 'GR', 'HK', 'HU', 'IS', 'IN',
#                   'ID', 'IE', 'IL', 'IT', 'JP', 'LT', 'MY', 'MX', 'NZ', 'NO', 'PA', 'PE', 'PL', 'PT', 'RO', 'RS', 'SG',
#                   'ZA', 'ES', 'SE', 'CH', 'TH', 'TR', 'UA']
#
##^^^^^^^^^^^^^^^^^^^^^^##
## TO LONG FOR FREE API ##
##########################


# short list for testing
country_code_lst_short =  ["SE", "GB", "BR", "HR", "IN"]
service_lst_short = ["netflix", "prime", "hbo", "disney"]

### Code

In [ ]:
# API key
api_key = input(f"Get key from https://rapidapi.com/movie-of-the-night-movie-of-the-night-default/api/streaming-availability/\nkey:")


# Get search input 
movie_or_series = search_movie_series()
keyword = search_input()

# Collect data 
df = colect_data(service_lst_short, country_code_lst_short, movie_or_series, keyword, api_key)

# Clean data
df_copy = clean_data_service(df.copy())
df_copy = add_missing_service(df_copy, service_lst_short)
df_copy = add_country(df_copy)
df_copy = clean_df(df_copy)

# Change df style
df_copy = df_styler(df_copy)

# Convert to HTML and oppen in browser
convert_to_HTML(df_copy)
    